In [77]:
import numpy as np
import snap
import matplotlib.pyplot as plt
import networkx as nx
%matplotlib inline

### 加载数据

In [78]:
G=snap.TUNGraph.Load(snap.TFIn('hw1-q2.graph'))
print("# of Nodes",G.GetNodes())
print("# of Edges",G.GetEdges())

# of Nodes 1589
# of Edges 2742


In [79]:
def vec2Numpy(v):
    r=np.empty(shape=(v.Len(),))
    for i,x in enumerate(v):r[i]=x
    return r
def similar(feature,target):
    
    n1=np.linalg.norm(feature,axis=1)
    n2=np.linalg.norm(target)
    assert n2>0
    result=feature.dot(target)/n1/n2
    #防止属性全为0的情况
    mask=np.where(n1==0)[0]
    result[mask]=0
    return result


def topKSimilar(feature,targetIndex,K):
    score=similar(feature,feature[targetIndex])
    rank=np.argsort(-score)[1:K+1]
    return rank

### 特征统计
* 每个顶点的度
* 每个顶点egonet的边
* 每个顶点egonet的对外边

In [82]:
def getDegree(G):
    v=snap.TIntV()
    snap.GetDegSeqV(G,v)
    return vec2Numpy(v)[:,np.newaxis]
def getEgonetFeature(G):
    egonet_features=np.empty(shape=(G.GetNodes(),2));
    for i,node in enumerate(G.Nodes()):
        neighbours=snap.TIntV()
        degree=node.GetDeg()
        for cn in range(degree):
            nid=node.GetNbrNId(cn)
            neighbours.Add(nid)
        neighbours.Add(node.GetId())
        egonet_features[i]=snap.GetEdgesInOut(G,neighbours)
    return egonet_features
def extreactBaseFeature(G):
    f1=getDegree(G)
    f23=getEgonetFeature(G)
    feature=np.concatenate((f1,f23),axis=1)
    return feature

In [97]:
basefeature=extreactBaseFeature(G)
topKSimilar(basefeature,9,5)

/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:10: RuntimeWarning: invalid value encountered in true_divide


array([ 415,  288,  286, 1336, 1054])

#### 测试代码
* 测试提取的基本特征是不是正确

In [133]:
def sanity_check_feature(feature):
    ids=np.random.choice(np.arange(len(feature)),100,False)
    ids=[1,2,3]
    for idd in ids:
        print(idd,type(int(idd)))
        f1,f2,f3=feature[int(idd)]
        n=G.GetNI(idd)
        deg=n.GetDeg()
        
        all_node_degree=deg
        subnodes=[n.GetId()]
        for d in range(deg):
            nid=n.GetNbrNId(d)
            subnodes.append(nid)
            all_node_degree+=G.GetNI(nid).GetDeg()
        
        _f1=deg
        subgraph=snap.GetSubGraph(G,snap.TIntV.GetV(*subnodes))
        _f2=subgraph.GetEdges()
        _f3=all_node_degree-2*_f2
        
        assert _f1==f1 and _f2==f2 and _f3==f3
def print_neigh(G,t):
    n=G.GetNI(t)
    deg=n.GetDeg()
    nei=[]
    for i in range(deg):
        nei.append(n.GetNbrNId(i))
    print("%d:"%t,nei)

In [132]:
sanity_check_feature(basefeature)

1 <class 'int'>
2 <class 'int'>
3 <class 'int'>


In [74]:
def print_neigh(G,t):
    n=G.GetNI(t)
    deg=n.GetDeg()
    nei=[]
    for i in range(deg):
        nei.append(n.GetNbrNId(i))
    print("%d:"%t,nei)

nnn=9
n=G.GetNI(nnn)
deg=n.GetDeg()
neibour=[]
dd=deg
for d in range(deg):
    nid=n.GetNbrNId(d)
    print_neigh(G,nid)
    neibour.append(nid)
    dd+=G.GetNI(nid).GetDeg()
neibour.append(nnn)
subnodes=snap.TIntV.GetV(*neibour)
subgraph=snap.GetSubGraph(G,subnodes)
print(dd,2*subgraph.GetEdges())

7: [8, 9, 10, 11]
10: [7, 9, 11]
11: [7, 9, 10]
1424: [9, 1425]
1425: [9, 1424]
1532: [9]
21 20


In [117]:
a=np.random.randint(0,2,size=2)

In [122]:
type(int(a[0]))

int

In [119]:
a

array([0, 1])

In [121]:
a[0]

0